# Preparation

## import tools

In [9]:
import os, sys, numpy as np
import histogram.hdf as hh
import mcvine, mcvine.components
from mcni.utils import conversion

In [114]:
sys.path.insert(0, os.path.expanduser('~/git/Qikr/notebooks/production'))
sys.path.insert(0, os.path.expanduser('~/git/Qikr'))

import plot_utils

In [11]:
from mcvine import run_script

In [12]:
workdir = os.path.expanduser("~/simulations/mcvine/qikr")
!mkdir -p {workdir}
%cd {workdir}

/SNS/users/m2d/simulations/mcvine/qikr


# QIKR simulation

In [109]:
%%time
import qikr
reload(qikr)

!rm -rf out-mcvine/

# quick sim
instrument = qikr.qikr(wl_0=1.)
neutrons = instrument.simulate(int(1e5),outputdir="out-mcvine", overwrite_datafiles=False, iteration_no=0)

Ballistic guide 1: 1.0001 m
Ballistic guide 2: 6.6482 m
Bandwidth chopper: 7.1849 m
End of instrument: 15.9909 m
CPU times: user 3.23 s, sys: 86.7 ms, total: 3.32 s
Wall time: 3.6 s


# Parallel simulation

In [101]:
%%file qikr_instrument.py
import sys
import os
sys.path.insert(0, os.path.expanduser('~/git/Qikr'))
import qikr
instrument = qikr.qikr()

Overwriting qikr_instrument.py


In [104]:
%%time
!rm -rf ./work_5e8_dec6/
run_script.run_mpi('./qikr_instrument.py', workdir='./work_5e8_dec6', ncount=int(5e8), overwrite_datafiles=False, nodes=10)


CPU times: user 8.06 ms, sys: 121 ms, total: 129 ms
Wall time: 51min 19s


In [ ]:
#%%time
#run_script.run_mpi('./qikr_instrument.py', 'work_1e7_dec5', ncount=1e1, nodes=10, overwrite_datafiles=True)

## Results

`neutrons` is a numpy array of shape `(N,10)`. `N` is number of neutrons.

The 10 floats for a neutron are (x,y,z,vx,vy,vz,s1,s2,t,prob)

In [112]:
from mcni import neutron_storage  as ns

work_dir = './work_1e7_dec6'

neutrons = ns.readneutrons_asnpyarr("%s/beam.neutrons" % work_dir)
pre_chopper = ns.readneutrons_asnpyarr("%s/beam_pre_chopper.neutrons" % work_dir)
post_chopper = ns.readneutrons_asnpyarr("%s/beam_post_chopper.neutrons" % work_dir)
print neutrons.shape

histo, bins = np.histogram(1000.0*neutrons.T[4]/neutrons.T[5], bins=400, weights=neutrons.T[-1])
plot_utils.plot1d([bins, histo], y_log=False)



wl_pre = 2*np.pi/conversion.v2k(pre_chopper.T[5])
wl_post = 2*np.pi/conversion.v2k(post_chopper.T[5])
#wl_pre = 2*np.pi/conversio.v2k(pre_chopper.T[-2])
#wl_post = 2*np.pi/conversion.v2k(post_chopper.T[-2])


histo_pre, bins = np.histogram(wl_pre, bins=400, range=[0,30], weights=pre_chopper.T[-1])
histo_post, bins = np.histogram(wl_post, bins=bins, range=[0,30], weights=post_chopper.T[-1])
#histo_pre, bins = np.histogram(wl_pre, bins=400, weights=pre_chopper.T[-1])
#histo_post, bins = np.histogram(wl_post, bins=bins, weights=post_chopper.T[-1])


accept = histo_post/histo_pre
bin_mid = [(bins[i+1]+bins[i])/2.0 for i in range(len(bins)-1)]

#plot_utils.plot1d([[bins, histo_pre],[bins, histo_post]], y_log=True)
plot_utils.plot1d([[bin_mid, accept],], y_log=False)

(4372013, 10)


/SNS/software/miniconda2/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:27: RuntimeWarning:

invalid value encountered in divide

